<a href="https://colab.research.google.com/github/alekrul/RL/blob/main/GridWorld_iterativePolicyIteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
SMALL_ENOUGH = 1e-3
gamma = 0.9
ACTION_SPACE = ('U','L','D','R')

In [ ]:
#rewards = {(0,3): 1 , (1,3): -1}
actions = {
    (0,0): ('D','L'),
    (0,1): ('R','L'),
    (0,2): ('R','D','L'),
    (1,0): ('D','U'),
    (1,2): ('D','U','R'),
    (2,0): ('U','R'),
    (2,1): ('R','L'),
    (2,2): ('U','L','R'),
    (2,3): ('L','U')

}

In [ ]:
#policy to be evaluated
  ### fixed policy ###
policy = {
    (2, 0): 'U',
    (1, 0): 'U',
    (0, 0): 'R',
    (0, 1): 'R',
    (0, 2): 'R',
    (1, 2): 'U',
    (2, 1): 'R',
    (2, 2): 'U',
    (2, 3): 'L',
}

In [ ]:
class Grid: # Environment
  def __init__(self, rows, cols, start):
    self.rows = rows
    self.cols = cols
    self.i = start[0]
    self.j = start[1]

  def set(self, rewards, actions):
    # rewards should be a dict of: (i, j): r (row, col): reward
    # actions should be a dict of: (i, j): A (row, col): list of possible actions
    self.rewards = rewards
    self.actions = actions

  def set_state(self, s):
    self.i = s[0]
    self.j = s[1]

  def current_state(self):
    return (self.i, self.j)

  def is_terminal(self, s):
    return s not in self.actions

  def get_next_state(self, s, a):
    # this answers: where would I end up if I perform action 'a' in state 's'?
    i, j = s[0], s[1]

    # if this action moves you somewhere else, then it will be in this dictionary
    if a in self.actions[(i, j)]:
      if a == 'U':
        i -= 1
      elif a == 'D':
        i += 1
      elif a == 'R':
        j += 1
      elif a == 'L':
        j -= 1
    return i, j

  def move(self, action):
    # check if legal move first
    if action in self.actions[(self.i, self.j)]:
      if action == 'U':
        self.i -= 1
      elif action == 'D':
        self.i += 1
      elif action == 'R':
        self.j += 1
      elif action == 'L':
        self.j -= 1
    # return a reward (if any)
    return self.rewards.get((self.i, self.j), 0)

  def undo_move(self, action):
    # these are the opposite of what U/D/L/R should normally do
    if action == 'U':
      self.i += 1
    elif action == 'D':
      self.i -= 1
    elif action == 'R':
      self.j -= 1
    elif action == 'L':
      self.j += 1
    # raise an exception if we arrive somewhere we shouldn't be
    # should never happen
    assert(self.current_state() in self.all_states())

  def game_over(self):
    # returns true if game is over, else false
    # true if we are in a state where no actions are possible
    return (self.i, self.j) not in self.actions

  def all_states(self):
    # possibly buggy but simple way to get all states
    # either a position that has possible next actions
    # or a position that yields a reward
    return set(self.actions.keys()) | set(self.rewards.keys())

In [ ]:
def standard_grid():
  # define a grid that describes the reward for arriving at each state
  # and possible actions at each state
  # the grid looks like this
  # x means you can't go there
  # s means start position
  # number means reward at that state
  # .  .  .  1
  # .  x  . -1
  # s  .  .  .
  g = Grid(3, 4, (2, 0))
  rewards = {(0, 3): 1, (1, 3): -1}
  actions = {
    (0, 0): ('D', 'R'),
    (0, 1): ('L', 'R'),
    (0, 2): ('L', 'D', 'R'),
    (1, 0): ('U', 'D'),
    (1, 2): ('U', 'D', 'R'),
    (2, 0): ('U', 'R'),
    (2, 1): ('L', 'R'),
    (2, 2): ('L', 'R', 'U'),
    (2, 3): ('L', 'U'),
  }
  g.set(rewards, actions)
  return g


In [ ]:
def print_values(V, g):
  for i in range(g.rows):
    print("---------------------------")
    for j in range(g.cols):
      v = V.get((i,j), 0)
      if v >= 0:
        print(" %.2f|" % v, end="")
      else:
        print("%.2f|" % v, end="") # -ve sign takes up an extra space
    print("")

In [ ]:
def print_policy(P, g):
  for i in range(g.rows):
    print("---------------------------")
    for j in range(g.cols):
      a = P.get((i,j), ' ')
      print("  %s  |" % a, end="")
    print("")


In [ ]:
transition_probs = {}
rewards = {}

grid = standard_grid()
for i in range(grid.rows):
  for j in range(grid.cols):
    s = (i, j)
    if not grid.is_terminal(s):
      for a in ACTION_SPACE:
        s2 = grid.get_next_state(s, a)
        transition_probs[(s, a, s2)] = 1
        if s2 in grid.rewards:
          rewards[(s, a, s2)] = grid.rewards[s2]


In [ ]:
print_policy(policy,grid)

---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  U  |     |
---------------------------
  U  |  R  |  U  |  L  |


In [ ]:
# repeat until convergence
def evaluate_deterministic_policy(grid, policy):
  V = {}
  for s in grid.all_states():
    V[s] = 0
  it = 0
  while True:
    biggest_change = 0

    for s in grid.all_states():
      if not grid.is_terminal(s):
        old_v = V[s]
        new_v = 0 # we will accumulate the answer
        for a in ACTION_SPACE:
          for s2 in grid.all_states():

            # action probability is deterministic
            action_prob = 1 if policy.get(s) == a else 0
            
            # reward is a function of (s, a, s'), 0 if not specified
            r = rewards.get((s, a, s2), 0)
            new_v += action_prob * transition_probs.get((s, a, s2), 0) * (r + gamma * V[s2])

          # after done getting the new value, update the value table
        V[s] = new_v
        biggest_change = max(biggest_change, np.abs(old_v - V[s]))

    print("iter:", it, "biggest_change:", biggest_change)
    print_values(V, grid)
    it += 1

    if biggest_change < SMALL_ENOUGH:
      break
  return V


In [ ]:
policy = {} #random policy to start the iteration and evaluation cycle
for s in grid.actions.keys():
  policy[s] = np.random.choice(ACTION_SPACE)

print_policy(policy, grid)

---------------------------
  U  |  L  |  L  |     |
---------------------------
  R  |     |  R  |     |
---------------------------
  R  |  L  |  U  |  U  |


In [ ]:
while True:
  V = evaluate_deterministic_policy(grid, policy)

  is_policy_converged = True
  for s in grid.actions.keys():
    old_a = policy[s]
    new_a = None
    best_value = float('-Inf')

    for a in ACTION_SPACE:
      v = 0
      for s2 in grid.all_states():
        r = rewards.get((s,a,s2),0)
        v += transition_probs.get((s,a,s2),0)*(r + gamma*V[s2])
      if v > best_value:
        best_value = v
        new_a = a
    policy[s] = new_a
    if new_a != old_a:
      is_policy_converged = False
  if is_policy_converged:
    break
print("values:")
print_values(V,grid)
print("policy:")
print_policy(policy, grid)


iter: 0 biggest_change: 1.0
---------------------------
 0.00| 0.00| 0.00| 0.00|
---------------------------
 0.00| 0.00|-1.00| 0.00|
---------------------------
 0.00| 0.00|-0.90|-1.00|
iter: 1 biggest_change: 0
---------------------------
 0.00| 0.00| 0.00| 0.00|
---------------------------
 0.00| 0.00|-1.00| 0.00|
---------------------------
 0.00| 0.00|-0.90|-1.00|
iter: 0 biggest_change: 1.0
---------------------------
 0.00| 0.00| 1.00| 0.00|
---------------------------
 0.00| 0.00| 0.00| 0.00|
---------------------------
 0.00| 0.00| 0.00| 0.00|
iter: 1 biggest_change: 0.9
---------------------------
 0.00| 0.00| 1.00| 0.00|
---------------------------
 0.00| 0.00| 0.90| 0.00|
---------------------------
 0.00| 0.00| 0.00| 0.00|
iter: 2 biggest_change: 0
---------------------------
 0.00| 0.00| 1.00| 0.00|
---------------------------
 0.00| 0.00| 0.90| 0.00|
---------------------------
 0.00| 0.00| 0.00| 0.00|
iter: 0 biggest_change: 1.0
---------------------------
 0.00| 0.00| 